In [1]:
import numpy as np
import torch
from deepymod_torch.library_function import library_new
from deepymod_torch.DeepMod import DeepMod, deepmod_type
from deepymod_torch.sparsity import scaling, threshold
from deepymod_torch.nn import create_deriv_data

import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

np.random.seed(40)

%load_ext autoreload
%autoreload 2

#torch.set_default_tensor_type('torch.cuda.FloatTensor')  # enable for gpu.

In [2]:
data = np.load('../data/processed/burgers.npy', allow_pickle=True).item()

In [3]:
X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))

In [4]:
number_of_samples = 1000

idx = np.random.permutation(y.size)
X_train = create_deriv_data(torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True), 2)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [5]:
library_config = {'type': library_new, 'poly_order': 2, 'diff_order': 2}
network_config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1}
model = DeepMod(network_config, library_config)

In [7]:
len(model(X_train))

3

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)  

In [9]:
model.train(X_train, y_train, optimizer, 100)

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
        100    100.00%               0s   4.22e-01   1.20e-01   3.02e-01   1.12e-04 

In [9]:
model.sparsity_mask_list

[tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])]

In [9]:
model.coeff_vector_list[0]

Parameter containing:
tensor([[0.4871],
        [0.5918],
        [0.5562],
        [0.0716],
        [0.3055],
        [0.1872],
        [0.2820],
        [0.4201],
        [0.1087]], requires_grad=True)

In [11]:
model.parameters()

<generator object Module.parameters at 0x7f494a80bd58>